In [56]:
# pip install fuzzywuzzy rapidfuzz metaphone time

ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import fuzz, process
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from rapidfuzz.distance import Levenshtein  # Faster than normal Levenshtein

import warnings
warnings.filterwarnings("ignore")


/opt/anaconda3/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [82]:
import time
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# Load Data
G = pd.read_csv("G.csv", delimiter="|", encoding="utf-8")
STrain = pd.read_csv("STrain.csv", delimiter="|", encoding="utf-8")

# Standardize Names (Preprocess to normalize variations)
def standardize_company_name(name):
    """Convert company names to a standardized format."""
    if pd.isnull(name):
        return ""
    
    name = name.lower().strip()
    replacements = {"llc": "", "gmbh": "", "inc": "", "s.a.": "", "sa": "", "ltd": ""}
    
    for short, full in replacements.items():
        name = name.replace(short, full)
    
    return name.strip()

# Apply standardization
G["clean_name"] = G["name"].apply(standardize_company_name)
STrain["clean_name"] = STrain["name"].apply(standardize_company_name)

# Convert to lists
ground_truth_names = G["clean_name"].tolist()
train_names = STrain["clean_name"].tolist()

# Precompute TF-IDF Vectors
vectorizer = TfidfVectorizer()
tfidf_g_matrix = vectorizer.fit_transform(ground_truth_names)  # Ground truth companies
tfidf_s_matrix = vectorizer.transform(train_names)  # External dataset

# Use Nearest Neighbors for Fast Matching
nn = NearestNeighbors(n_neighbors=1, metric='cosine', n_jobs=-1)  # Multi-threaded search
nn.fit(tfidf_g_matrix)



NearestNeighbors(metric='cosine', n_jobs=-1, n_neighbors=1)

In [84]:
# Start timing
start_time = time.time()

# Find nearest match for each row in STrain
distances, best_match_indices = nn.kneighbors(tfidf_s_matrix, return_distance=True)

# Convert cosine distances to similarity scores
best_match_scores = 1 - distances.flatten()  # Higher = better match
best_match_names = [ground_truth_names[idx] for idx in best_match_indices.flatten()]

# ** Assign company_id Efficiently**
G_dict = dict(zip(G["clean_name"], G["company_id"]))  # Precompute dictionary for fast lookup
STrain["matched_name"] = best_match_names
STrain["best_match_score"] = best_match_scores
STrain["company_id_pred"] = STrain["matched_name"].map(G_dict).fillna(-1).astype(int)  # Assign IDs or -1 if not found

# End timing
end_time = time.time()


# Print execution time
total_time = (end_time - start_time) / 60  # Convert to minutes
print(f" Optimized Execution Time: {total_time:.2f} minutes")

✅ Optimized Execution Time: 13.37 minutes


In [86]:
STrain

,train_index,name,company_id,clean_name,matched_name,best_match_score,company_id_pred
0,0,ATRION Immo bilien & Co. KG,-1,atrion immo bilien & co. kg,atrion immobilien verwaltung,0.585725,250537
1,1,MyTyme Inve stments Inc,356624,mytyme inve stments,mytyme investments,0.654939,356624
2,2,Financial USI.,510805,financial usi.,best usi portfolio,0.721416,80311
3,3,FlexShares Trust - FlexShares Morningstar Emer...,523467,flexshares trust - flexshares morningstar emer...,flexshares trust - flexshares morningstar emer...,0.888245,523467
4,4,Health Sinai SF,231108,health sinai sf,sinai health system foundation,0.679153,231108
...,...,...,...,...,...,...,...
99995,99995,DELTA FOREX Fund LIMITED,346113,delta forex fund limited,delta forex limited,0.967177,346113
99996,99996,"Inte. IV Vector, L.P.",-1,"inte. iv vector, l.p.","""vector - s.p.a.""",0.657187,185355
99997,99997,LE AQUI. SOCIETA' A RESPONSABILITA' LIMITATA,305436,le aqui. societa' a responbilita' limitata,v.a.r.m.o. societa' a responbilita' limitata,0.593305,451543
99998,99998,"DP Lease Mezz Company I Holding Company, LLC",585299,"dp lease mezz company i holding company,","dp lease mezz i holding company,",0.967879,585299


In [13]:
import pandas as pd

def calculate_cost(row):
    """
    Compute the cost function per row.
    Args:
        row (pd.Series): A single row containing 'company_id' (true) and 'company_id_pred' (predicted).
    
    Returns:
        cost (int): Cost for this row based on the given function.
    """
    true_id = row["company_id"]
    pred_id = row["company_id_pred"]
    
    if true_id == pred_id:
        return 0  # Correct match (Best Case)
    elif pred_id == -1:
        return 1  # Incorrectly classified as "not in G"
    else:
        return 5  # Incorrectly matched to another company (Worst Case)


# Apply cost function to each row
STrain["cost_function"] = STrain.apply(calculate_cost, axis=1)

# Save the new DataFrame with cost function
STrain.to_csv("STrain_with_cost_function.csv", sep="|", index=False)

# Display sample results
print(STrain.head())

   train_index                                               name  company_id  \
0            0                        ATRION Immo bilien & Co. KG          -1   
1            1                            MyTyme Inve stments Inc      356624   
2            2                                     Financial USI.      510805   
3            3  FlexShares Trust - FlexShares Morningstar Emer...      523467   
4            4                                    Health Sinai SF      231108   

                                          clean_name  \
0                        atrion immo bilien & co. kg   
1                            mytyme inve stments inc   
2                                     financial usi.   
3  flexshares trust - flexshares morningstar emer...   
4                                    health sinai sf   

                                       filtered_name  \
0                                 atrion immo bilien   
1                            mytyme inve stments inc   
2       

In [94]:
# Count occurrences of each cost value (0, 1, 5)
cost_counts = STrain["cost_function"].value_counts().sort_index()

# Print the counts
print(" Cost Function Occurrences:")
for cost, count in cost_counts.items():
    print(f"Cost {cost}: {count} occurrences")

# Optional: Convert to dictionary for further use
cost_distribution = cost_counts.to_dict()
print("\n Cost Distribution Dictionary:", cost_distribution)

✅ Cost Function Occurrences:
Cost 0: 37320 occurrences
Cost 5: 62680 occurrences

🚀 Cost Distribution Dictionary: {0: 37320, 5: 62680}


# Second trail

In [96]:
import time
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from metaphone import doublemetaphone

# Load Data
G = pd.read_csv("G.csv", delimiter="|", encoding="utf-8")
STrain = pd.read_csv("STrain.csv", delimiter="|", encoding="utf-8")

# Standardize Names
def standardize_company_name(name):
    """Preprocess company names by normalizing them."""
    if pd.isnull(name):
        return ""

    name = name.lower().strip()
    replacements = {"llc": "", "gmbh": "", "inc": "", "s.a.": "", "sa": "", "ltd": ""}
    for short, full in replacements.items():
        name = name.replace(short, full)

    return name.strip()

G["clean_name"] = G["name"].apply(standardize_company_name)
STrain["clean_name"] = STrain["name"].apply(standardize_company_name)

# Convert names into lists
ground_truth_names = G["clean_name"].tolist()
train_names = STrain["clean_name"].tolist()

# **Precompute TF-IDF Vectors**
vectorizer = TfidfVectorizer(stop_words="english", min_df=2, max_df=0.85)  # Remove common words
tfidf_g_matrix = vectorizer.fit_transform(ground_truth_names)
tfidf_s_matrix = vectorizer.transform(train_names)

# ** Use Nearest Neighbors with More Candidates**
nn = NearestNeighbors(n_neighbors=3, metric='cosine', n_jobs=-1)  # Multi-threaded
nn.fit(tfidf_g_matrix)

# Start timing
start_time = time.time()

# ** Find Top 3 Matches**
distances, best_match_indices = nn.kneighbors(tfidf_s_matrix, return_distance=True)

# Convert cosine distances to similarity scores
best_match_scores = 1 - distances  # Higher = better match

# Phonetic Matching for Edge Cases
def phonetic_match(query, candidates):
    """Find the best phonetic match."""
    query_code = doublemetaphone(query)[0]
    for name in candidates:
        if doublemetaphone(name)[0] == query_code:
            return name
    return None  # No phonetic match

# ** Apply Matching Rules**
final_matches = []
final_scores = []
for i, match_list in enumerate(best_match_indices):
    top_match = None
    top_score = 0
    for j, idx in enumerate(match_list):
        if best_match_scores[i][j] > 0.7:  # Only accept strong matches
            top_match = ground_truth_names[idx]
            top_score = best_match_scores[i][j]
            break  # Stop at first valid match
    
    # If no strong match, try phonetic matching
    if not top_match:
        top_match = phonetic_match(train_names[i], [ground_truth_names[idx] for idx in match_list])

    final_matches.append(top_match)
    final_scores.append(top_score)

# Assign company_id Efficiently
G_dict = dict(zip(G["clean_name"], G["company_id"]))  # Precompute dictionary for fast lookup
STrain["matched_name"] = final_matches
STrain["best_match_score"] = final_scores
STrain["company_id_pred"] = STrain["matched_name"].map(G_dict).fillna(-1).astype(int)  # Assign IDs or -1 if not found

# End timing
end_time = time.time()

# Print execution time
total_time = (end_time - start_time) / 60  # Convert to minutes
print(f" Optimized Execution Time: {total_time:.2f} minutes")

✅ Optimized Execution Time: 20.40 minutes


In [98]:
# Apply cost function to each row
STrain["cost_function"] = STrain.apply(calculate_cost, axis=1)
# Count occurrences of each cost value (0, 1, 5)
cost_counts = STrain["cost_function"].value_counts().sort_index()

# Print the counts
print("Cost Function Occurrences:")
for cost, count in cost_counts.items():
    print(f"Cost {cost}: {count} occurrences")

# Optional: Convert to dictionary for further use
cost_distribution = cost_counts.to_dict()
print("Cost Distribution Dictionary:", cost_distribution)


✅ Cost Function Occurrences:
Cost 0: 35856 occurrences
Cost 1: 14367 occurrences
Cost 5: 49777 occurrences

🚀 Cost Distribution Dictionary: {0: 35856, 1: 14367, 5: 49777}


# Next-Level Optimizations

1. Boost Precision by Adjusting TF-IDF Weights
- Remove stopwords that confuse matching.
- Lower min_df to capture rare company names.
- Increase ngram range to allow better name token matching.

2. Strengthen Threshold Criteria
- Instead of using one fixed threshold (0.7), dynamically adjust it.
- Use best_match_score > (0.75 + 0.1 * name_length_factor).

3. Check for Exact Name Substrings
- If a perfect substring match exists, prioritize it over TF-IDF.

4. Phonetic Matching with Fuzzy Filtering
- Use doublemetaphone() + fuzzy ratio to confirm phonetic matches.

In [101]:
import time
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from metaphone import doublemetaphone
from rapidfuzz import fuzz, process

# Load Data
G = pd.read_csv("G.csv", delimiter="|", encoding="utf-8")
STrain = pd.read_csv("STrain.csv", delimiter="|", encoding="utf-8")

# Standardize Names
def standardize_company_name(name):
    """Preprocess company names by normalizing them."""
    if pd.isnull(name):
        return ""

    name = name.lower().strip()
    replacements = {"llc": "", "gmbh": "", "inc": "", "s.a.": "", "sa": "", "ltd": ""}
    for short, full in replacements.items():
        name = name.replace(short, full)

    return name.strip()

G["clean_name"] = G["name"].apply(standardize_company_name)
STrain["clean_name"] = STrain["name"].apply(standardize_company_name)

# Convert names into lists
ground_truth_names = G["clean_name"].tolist()
train_names = STrain["clean_name"].tolist()

# ** Optimized TF-IDF Vectorization**
vectorizer = TfidfVectorizer(stop_words="english", min_df=1, max_df=0.85, ngram_range=(1,2))  
tfidf_g_matrix = vectorizer.fit_transform(ground_truth_names)
tfidf_s_matrix = vectorizer.transform(train_names)

# ** Use Nearest Neighbors with More Candidates**
nn = NearestNeighbors(n_neighbors=5, metric='cosine', n_jobs=-1)  # Multi-threaded
nn.fit(tfidf_g_matrix)



NearestNeighbors(metric='cosine', n_jobs=-1)

In [102]:
# Start timing
start_time = time.time()

# ** Find Top 5 Matches**
distances, best_match_indices = nn.kneighbors(tfidf_s_matrix, return_distance=True)

# Convert cosine distances to similarity scores
best_match_scores = 1 - distances  # Higher = better match

# ** Phonetic Matching + Exact Match**
def phonetic_match(query, candidates):
    """Find the best phonetic match."""
    query_code = doublemetaphone(query)[0]
    best_match = None
    best_score = 0
    
    for name in candidates:
        name_code = doublemetaphone(name)[0]
        if query_code == name_code:  # Exact phonetic match
            return name
        score = fuzz.token_sort_ratio(query, name)
        if score > best_score:
            best_score = score
            best_match = name if best_score > 85 else None  # Threshold
    
    return best_match

# ** Apply Optimized Matching Rules**
final_matches = []
final_scores = []
for i, match_list in enumerate(best_match_indices):
    top_match = None
    top_score = 0
    
    for j, idx in enumerate(match_list):
        similarity_score = best_match_scores[i][j]
        length_factor = min(len(train_names[i]) / 50, 1)  # Adjust based on name length
        threshold = 0.75 + (0.1 * length_factor)  # Dynamic threshold

        if similarity_score > threshold:  
            top_match = ground_truth_names[idx]
            top_score = similarity_score
            break  # Stop at first valid match
    
    # If no strong match, try phonetic matching
    if not top_match:
        top_match = phonetic_match(train_names[i], [ground_truth_names[idx] for idx in match_list])

    # If still no match, assign -1
    if not top_match:
        top_match = None
        top_score = -1

    final_matches.append(top_match)
    final_scores.append(top_score)

# ** Assign company_id Efficiently**
G_dict = dict(zip(G["clean_name"], G["company_id"]))  # Precompute dictionary for fast lookup
STrain["matched_name"] = final_matches
STrain["best_match_score"] = final_scores
STrain["company_id_pred"] = STrain["matched_name"].map(G_dict).fillna(-1).astype(int)  # Assign IDs or -1 if not found

# End timing
end_time = time.time()

# Print execution time
total_time = (end_time - start_time) / 60  # Convert to minutes
print(f" Final Optimized Execution Time: {total_time:.2f} minutes")


✅ Final Optimized Execution Time: 11.70 minutes


In [104]:
# Apply cost function to each row
STrain["cost_function"] = STrain.apply(calculate_cost, axis=1)
# Count occurrences of each cost value (0, 1, 5)
cost_counts = STrain["cost_function"].value_counts().sort_index()

# Print the counts
print(" Cost Function Occurrences:")
for cost, count in cost_counts.items():
    print(f"Cost {cost}: {count} occurrences")

# Optional: Convert to dictionary for further use
cost_distribution = cost_counts.to_dict()
print("\n Cost Distribution Dictionary:", cost_distribution)


✅ Cost Function Occurrences:
Cost 0: 52195 occurrences
Cost 1: 31292 occurrences
Cost 5: 16513 occurrences

🚀 Cost Distribution Dictionary: {0: 52195, 1: 31292, 5: 16513}


| **Cost Type**                   | **Previous Optimized Model** | **Latest Optimized Model** | **Improvement**       |
|---------------------------------|-----------------------------|---------------------------|-----------------------|
|  **Correct Matches (Cost 0)**   | 52,195                      | 51,435                    | -760 (slight decrease) |
|  **Incorrect as `-1` (Cost 1)** | 31,292                      | 30,453                    | -839 (Better!)       |
|  **Incorrect Matches (Cost 5)** | 16,513                      | 18,112                    | +1,599 (Slight increase) |

New Optimized Code
- Keeps all previous improvements.
- Adds a secondary fuzzy match for low-confidence cases (-1 assignments).

In [109]:
import time
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from metaphone import doublemetaphone
from rapidfuzz import fuzz, process

STrain = pd.read_csv("STrain.csv", delimiter="|", encoding="utf-8")

# Start timing
start_time = time.time()

# **Find Top 5 Matches**
distances, best_match_indices = nn.kneighbors(tfidf_s_matrix, return_distance=True)

# Convert cosine distances to similarity scores
best_match_scores = 1 - distances  # Higher = better match

# ** Phonetic Matching + Exact Match**
def phonetic_match(query, candidates):
    """Find the best phonetic match."""
    query_code = doublemetaphone(query)[0]
    best_match = None
    best_score = 0
    
    for name in candidates:
        name_code = doublemetaphone(name)[0]
        if query_code == name_code:  # Exact phonetic match
            return name
        score = fuzz.token_sort_ratio(query, name)
        if score > best_score:
            best_score = score
            best_match = name if best_score > 85 else None  # Threshold
    
    return best_match

# ** Apply Optimized Matching Rules**
final_matches = []
final_scores = []
low_confidence_cases = []

for i, match_list in enumerate(best_match_indices):
    top_match = None
    top_score = 0
    
    for j, idx in enumerate(match_list):
        similarity_score = best_match_scores[i][j]
        length_factor = min(len(train_names[i]) / 50, 1)  # Adjust based on name length
        threshold = 0.75 + (0.1 * length_factor)  # Dynamic threshold

        if similarity_score > threshold:  
            top_match = ground_truth_names[idx]
            top_score = similarity_score
            break  # Stop at first valid match
    
    # If no strong match, try phonetic matching
    if not top_match:
        top_match = phonetic_match(train_names[i], [ground_truth_names[idx] for idx in match_list])

    # If still no match, mark for secondary fuzzy matching
    if not top_match:
        top_match = None
        top_score = -1
        if 0.6 <= similarity_score <= 0.7:  # If borderline case, store for secondary fuzzy matching
            low_confidence_cases.append((i, train_names[i]))

    final_matches.append(top_match)
    final_scores.append(top_score)

# ** Secondary Matching for `Cost = 1` Cases**
def secondary_fuzzy_match(query, candidates):
    """Find a possible match for items that were incorrectly classified as -1."""
    result = process.extractOne(query, candidates, scorer=fuzz.partial_ratio)

    if result:  # Ensure result is not None
        match = result[0]  # Extract only the best match
        score = result[1]  # Extract only the similarity score
        return match if score >= 75 else None  # Only accept matches with score ≥ 75
    
    return None  # No suitable match found

# ** Apply Secondary Matching**
for i, query in low_confidence_cases:
    new_match = secondary_fuzzy_match(query, ground_truth_names)
    if new_match:
        final_matches[i] = new_match
        final_scores[i] = 0.65  # Assign a moderate confidence score

# ** Assign company_id Efficiently**
G_dict = dict(zip(G["clean_name"], G["company_id"]))  # Precompute dictionary for fast lookup
STrain["matched_name"] = final_matches
STrain["best_match_score"] = final_scores
STrain["company_id_pred"] = STrain["matched_name"].map(G_dict).fillna(-1).astype(int)  # Assign IDs or -1 if not found

# End timing
end_time = time.time()

# Print execution time
total_time = (end_time - start_time) / 60  # Convert to minutes
print(f" Final Optimized Execution Time: {total_time:.2f} minutes")

 Final Optimized Execution Time: 13.82 minutes


In [110]:
# Apply cost function to each row
STrain["cost_function"] = STrain.apply(calculate_cost, axis=1)
# Count occurrences of each cost value (0, 1, 5)
cost_counts = STrain["cost_function"].value_counts().sort_index()

# Print the counts
print("Cost Function Occurrences:")
for cost, count in cost_counts.items():
    print(f"Cost {cost}: {count} occurrences")

# Optional: Convert to dictionary for further use
cost_distribution = cost_counts.to_dict()
print("\n Cost Distribution Dictionary:", cost_distribution)


Cost Function Occurrences:
Cost 0: 51435 occurrences
Cost 1: 30453 occurrences
Cost 5: 18112 occurrences

 Cost Distribution Dictionary: {0: 51435, 1: 30453, 5: 18112}


| **Cost Type**                   | **Previous Optimized Model** | **Latest Optimized Model** | **Improvement**       |
|---------------------------------|-----------------------------|---------------------------|-----------------------|
|  **Correct Matches (Cost 0)**   | 52,195                      | 51,435                    | -760 (slight decrease) |
|  **Incorrect as `-1` (Cost 1)** | 31,292                      | 30,453                    | -839 (Better!)       |
|  **Incorrect Matches (Cost 5)** | 16,513                      | 18,112                    | +1,599 (Slight increase) |

Now, we prevent bad matches by:
- Ensuring fuzzy matches are only accepted if score ≥ 80.
- Filtering out low-confidence phonetic matches.


In [113]:
import time
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from metaphone import doublemetaphone
from rapidfuzz import fuzz, process

STrain = pd.read_csv("STrain.csv", delimiter="|", encoding="utf-8")

# Start timing
start_time = time.time()

# **Find Top 5 Matches**
distances, best_match_indices = nn.kneighbors(tfidf_s_matrix, return_distance=True)

# Convert cosine distances to similarity scores
best_match_scores = 1 - distances  # Higher = better match

# ** Phonetic Matching + Exact Match**
def phonetic_match(query, candidates):
    """Find the best phonetic match."""
    query_code = doublemetaphone(query)[0]
    best_match = None
    best_score = 0
    
    for name in candidates:
        name_code = doublemetaphone(name)[0]
        if query_code == name_code:  # Exact phonetic match
            return name
        score = fuzz.token_sort_ratio(query, name)
        if score > best_score:
            best_score = score
            best_match = name if best_score > 85 else None  # Threshold
    
    return best_match

# ** Apply Optimized Matching Rules**
final_matches = []
final_scores = []
low_confidence_cases = []

for i, match_list in enumerate(best_match_indices):
    top_match = None
    top_score = 0
    
    for j, idx in enumerate(match_list):
        similarity_score = best_match_scores[i][j]
        length_factor = min(len(train_names[i]) / 50, 1)  # Adjust based on name length
        threshold = 0.75 + (0.1 * length_factor)  # Dynamic threshold

        if similarity_score > threshold:  
            top_match = ground_truth_names[idx]
            top_score = similarity_score
            break  # Stop at first valid match
    
    # If no strong match, try phonetic matching
    if not top_match:
        top_match = phonetic_match(train_names[i], [ground_truth_names[idx] for idx in match_list])

    # If still no match, mark for secondary fuzzy matching
    if not top_match:
        top_match = None
        top_score = -1
        if 0.6 <= similarity_score <= 0.7:  # If borderline case, store for secondary fuzzy matching
            low_confidence_cases.append((i, train_names[i]))

    final_matches.append(top_match)
    final_scores.append(top_score)

# ** Secondary Matching for `Cost = 1` Cases**
def secondary_fuzzy_match(query, candidates):
    """Find a possible match for items that were incorrectly classified as -1."""
    result = process.extractOne(query, candidates, scorer=fuzz.partial_ratio)

    if result:  # Ensure result is not None
        match = result[0]  # Extract only the best match
        score = result[1]  # Extract only the similarity score
        return match if score >= 75 else None  # Only accept matches with score ≥ 75
    
    return None  # No suitable match found

# ** Apply Secondary Matching (Fix Low-Confidence Matches)**
for i, query in low_confidence_cases:
    new_match = secondary_fuzzy_match(query, ground_truth_names)
    if new_match:
        # **Check Confidence Before Accepting**
        match_score = fuzz.token_sort_ratio(query, new_match)
        if match_score >= 80:  # Only accept matches with strong confidence
            final_matches[i] = new_match
            final_scores[i] = match_score / 100  # Normalize score (0 to 1)

# ** Assign company_id Efficiently**
G_dict = dict(zip(G["clean_name"], G["company_id"]))  # Precompute dictionary for fast lookup
STrain["matched_name"] = final_matches
STrain["best_match_score"] = final_scores
STrain["company_id_pred"] = STrain["matched_name"].map(G_dict).fillna(-1).astype(int)  # Assign IDs or -1 if not found

# End timing
end_time = time.time()

# Print execution time
total_time = (end_time - start_time) / 60  # Convert to minutes
print(f" Final Optimized Execution Time: {total_time:.2f} minutes")

 Final Optimized Execution Time: 13.45 minutes


In [114]:
# Apply cost function to each row
STrain["cost_function"] = STrain.apply(calculate_cost, axis=1)
# Count occurrences of each cost value (0, 1, 5)
cost_counts = STrain["cost_function"].value_counts().sort_index()

# Print the counts
print("Cost Function Occurrences:")
for cost, count in cost_counts.items():
    print(f"Cost {cost}: {count} occurrences")

# Optional: Convert to dictionary for further use
cost_distribution = cost_counts.to_dict()
print("\n Cost Distribution Dictionary:", cost_distribution)


Cost Function Occurrences:
Cost 0: 52191 occurrences
Cost 1: 31286 occurrences
Cost 5: 16523 occurrences

 Cost Distribution Dictionary: {0: 52191, 1: 31286, 5: 16523}


| **Cost Type**                   | **Previous Optimized Model** | **Final Optimized Model** | **Improvement**       |
|---------------------------------|-----------------------------|--------------------------|-----------------------|
|  **Correct Matches (Cost 0)**   | 51,435                      | **52,191**               | **+756 (Better!)** |
|  **Incorrect as `-1` (Cost 1)** | 30,453                      | **31,286**               | **+833 (Small increase)** |
|  **Incorrect Matches (Cost 5)** | 18,112                      | **16,523**               | **-1,589 (Big Reduction!)**  |

In [121]:
STrain

,train_index,name,company_id,matched_name,best_match_score,company_id_pred,cost_function
0,0,ATRION Immo bilien & Co. KG,-1,None,-1.000000,-1,0
1,1,MyTyme Inve stments Inc,356624,mytyme investments,0.000000,356624,0
2,2,Financial USI.,510805,None,-1.000000,-1,1
3,3,FlexShares Trust - FlexShares Morningstar Emer...,523467,flexshares trust - flexshares morningstar us m...,0.858506,532081,5
4,4,Health Sinai SF,231108,None,-1.000000,-1,1
...,...,...,...,...,...,...,...
99995,99995,DELTA FOREX Fund LIMITED,346113,delta forex limited,0.000000,346113,0
99996,99996,"Inte. IV Vector, L.P.",-1,None,-1.000000,-1,0
99997,99997,LE AQUI. SOCIETA' A RESPONSABILITA' LIMITATA,305436,f.a.p. - societa' a responbilita' limitata,0.000000,133515,5
99998,99998,"DP Lease Mezz Company I Holding Company, LLC",585299,"dp lease mezz i holding company,",0.000000,585299,0


In [123]:
G

,company_id,name,clean_name
0,634022,PRIMCOM SA,primcom
1,324497,The David Isaacs Fund,the david iacs fund
2,280848,Bramor Enterprises Limited,bramor enterprises limited
3,432662,NAVEXIM S.A.,navexim
4,524224,Magal Group SA,magal group
...,...,...,...
450251,614780,GL Europe Select Opportunities Investments Sarl,gl europe select opportunities investments rl
450252,26499,JACMA S.A. - SPF en Liquidation,jacma - spf en liquidation
450253,581457,GL EUROPE LUXEMBOURG SARL,gl europe luxembourg rl
450254,401977,MAX MARA INTERNATIONAL S.A.,max mara international


In [125]:
STrain = STrain.merge(G[['company_id', 'name', 'clean_name']], 
                      left_on='company_id_pred', 
                      right_on='company_id', 
                      how='left', 
                      suffixes=('', '_g'))
STrain[STrain["cost_function"] == 5].head(50)

In [148]:
STrain[STrain["cost_function"] == 5].head(50)

,train_index,name,company_id,matched_name,best_match_score,company_id_pred,cost_function,company_id_g,name_g,clean_name
3,3,FlexShares Trust - FlexShares Morningstar Emergingmarkets Factor Tilt Index Fund,523467,flexshares trust - flexshares morningstar us market factor tilt index fund,0.858506,532081,5,532081.0,FlexShares Trust - FlexShares Morningstar US Market Factor Tilt Index Fund,flexshares trust - flexshares morningstar us market factor tilt index fund
6,6,"LLC TEBS Fund II, ATAX",277891,"atax mbs fund i,",0.000000,324784,5,324784.0,"ATAX MBS Fund I, LLC","atax mbs fund i,"
9,9,"Tatra Asset Management, spárv. spol., a.s., depoz. o.p.f.",589052,"tatra asset management, správ. spol., a.s., dlhopisový o.p.f.",0.000000,373865,5,373865.0,"Tatra Asset Management, správ. spol., a.s., dlhopisový o.p.f.","tatra asset management, správ. spol., a.s., dlhopisový o.p.f."
12,12,Inteventor ApS,-1,h. w-w aps,1.000000,164577,5,164577.0,H. W-W ApS,h. w-w aps
15,15,RA iWeibull AB,253824,co.ra. s.p.a.,0.886437,612937,5,612937.0,CO.RA. S.P.A.,co.ra. s.p.a.
19,19,CAC,214855,"l/n cac,",1.000000,634653,5,634653.0,"L/N CAC, LLC","l/n cac,"
25,25,Services Comitno,-1,a.h.t. services,1.000000,387528,5,387528.0,A.H.T. Services GmbH,a.h.t. services
27,27,REPULIC ASSET MANAGEMENT,-1,s.t.a.r. asset management - s.r.l.,1.000000,513638,5,513638.0,S.T.A.R. ASSET MANAGEMENT - S.R.L.,s.t.a.r. asset management - s.r.l.
29,29,HSBC SPEcCIAL EUROPE SITU.,-1,in situ,0.803722,467769,5,467769.0,IN SITU S.A.,in situ
30,30,SONORA LLC Export,-1,sonora system s.l.,0.827719,478767,5,478767.0,SONORA SYSTEM S.L.,sonora system s.l.


In [135]:
pd.set_option("display.max_colwidth", None)
print(G[G["company_id"].isin([523467, 532081])])

        company_id  \
101142      532081   
432988      523467   

                                                                                     name  \
101142         FlexShares Trust - FlexShares Morningstar US Market Factor Tilt Index Fund   
432988  FlexShares Trust - FlexShares Morningstar Emerging Markets Factor Tilt Index Fund   

                                                                               clean_name  
101142         flexshares trust - flexshares morningstar us market factor tilt index fund  
432988  flexshares trust - flexshares morningstar emerging markets factor tilt index fund  


Why This is Happening
- Both names start with “FlexShares Trust - FlexShares Morningstar”.
- TF-IDF is overweighing common words like "FlexShares", "Trust", "Morningstar".
- The key distinguishing words (US Market Factor vs Emerging Markets Factor) are at the end, which reduces their importance.


Modify TF-IDF to prioritize the last few words in long names.



In [137]:
print(G[G["company_id"].isin([305436, 133515])])

        company_id  \
199736      305436   
266623      133515   

                                                                 name  \
199736  LE AQUILE BUSINESS CENTER SOCIETA' A RESPONSABILITA' LIMITATA   
266623                   F.A.P. - SOCIETA' A RESPONSABILITA' LIMITATA   

                                                         clean_name  
199736  le aquile business center societa' a responbilita' limitata  
266623                   f.a.p. - societa' a responbilita' limitata  


Why This is Happening
- "Societa' a Responsabilita' Limitata" is the legal structure (like "LLC" or "Inc.").
- The real company name is “LE AQUILE BUSINESS CENTER” vs “F.A.P.”, but the model treats them as similar.
- TF-IDF and Fuzzy Matching are incorrectly focusing on "Societa' a Responsabilita' Limitata", ignoring the unique parts.

| Issue                                                | Root Cause                                           | Fix                                      |
|------------------------------------------------------|------------------------------------------------------|------------------------------------------|
| Similar company names (FlexShares Trust...) matched incorrectly | TF-IDF prioritizes the start of names, ignoring key differences | Increase n-gram range & weight distinct words |
| Legal structures removed incorrectly (LLC, S.P.A., etc.) | Over-simplification of names during preprocessing | Keep meaningful legal terms |
| Incorrect fuzzy matches (e.g., SAqGIL DISTRIBUTION → H2O DISTRIBUTION) | Fuzzy logic prioritizes common words | Apply stricter word overlap filtering |
| Companies with similar abbreviations matched incorrectly (p s. partners → P.S. Partners, LLC) | Fuzzy matching ignores key contextual differences | Check Jaccard similarity + exact word overlap |

## There are still incorrect matches (Cost = 5) due to:
1.	Overmatching Common Words (e.g., "distribution", "logistics", "group")
- These words appear frequently and cause incorrect pairings.
- Removing them before the TF-IDF and matching steps will improve accuracy.

2.	Low Confidence Matches Still Causing Errors
- The secondary fuzzy matching currently accepts matches with score ≥ 75, which may still be too low.
- We should only accept matches with score ≥ 85 and word overlap ≥ 70%.

In [9]:
import time
import pandas as pd
import numpy as np
import dask.dataframe as dd
from joblib import Parallel, delayed
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from metaphone import doublemetaphone
from rapidfuzz import fuzz, process
from collections import Counter

# Load Data
G = pd.read_csv("G.csv", delimiter="|", encoding="utf-8")
STrain = pd.read_csv("STrain.csv", delimiter="|", encoding="utf-8")

# Start timing
start_time = time.time()

# ** Step 1: Name Standardization (Vectorized for Speed) **
def standardize_company_name(name):
    if pd.isnull(name): return ""
    name = name.lower().strip()
    legal_terms = ["llc", "inc", "ltd", "s.p.a.", "s.r.l."]
    for term in legal_terms:
        if name.endswith(term):
            continue  
        name = name.replace(term, "").strip()
    return name

G["clean_name"] = G["name"].map(standardize_company_name)
STrain["clean_name"] = STrain["name"].map(standardize_company_name)

# ** Step 2: Remove High-Frequency Words **
def get_high_freq_words(names, top_n=15):
    words = " ".join(names).split()
    word_counts = Counter(words)
    return [word for word, _ in word_counts.most_common(top_n)]

high_freq_words = get_high_freq_words(G["clean_name"], top_n=15)

def filter_common_words(name, common_words):
    return " ".join([word for word in name.split() if word not in common_words])

G["filtered_name"] = G["clean_name"].map(lambda x: filter_common_words(x, high_freq_words))
STrain["filtered_name"] = STrain["clean_name"].map(lambda x: filter_common_words(x, high_freq_words))

# ** Step 3: TF-IDF Vectorization (Faster) **
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1, 3), max_features=8_000)
tfidf_g_matrix = vectorizer.fit_transform(G["filtered_name"])
tfidf_s_matrix = vectorizer.transform(STrain["filtered_name"])

# ** Step 4: Nearest Neighbors Matching (Reduced `n_neighbors=3`) **
nn = NearestNeighbors(n_neighbors=3, metric="cosine", n_jobs=-1)
nn.fit(tfidf_g_matrix)

distances, best_match_indices = nn.kneighbors(tfidf_s_matrix, return_distance=True)
best_match_scores = 1 - distances  

# ** Step 5: Optimize Phonetic Matching with Caching & Parallel Processing **
phonetic_cache = {}

def get_phonetic_code(name):
    """Get Metaphone code for a given name, using a cache to speed up computation."""
    if name in phonetic_cache:
        return phonetic_cache[name]
    code = doublemetaphone(name)[0]
    phonetic_cache[name] = code
    return code

def phonetic_match(query, candidates):
    """Match based on phonetics using Metaphone with a reduced set of candidates."""
    query_code = get_phonetic_code(query)
    for name in candidates:
        if get_phonetic_code(name) == query_code:
            return name
    return None

# ** Apply Phonetic Matching Only on Nearest Neighbors, Using Parallel Processing **
parallel_phonetic_matches = Parallel(n_jobs=-1)(delayed(phonetic_match)(
    STrain["filtered_name"].iloc[i],  
    [G["filtered_name"].iloc[idx] for idx in best_match_indices[i]]
) for i in range(len(STrain)))

# ** Step 6: Faster Fuzzy Matching **
def secondary_fuzzy_match(query, candidates):
    """Apply fuzzy matching with stricter acceptance rules."""
    result = process.extractOne(query, candidates, scorer=fuzz.token_sort_ratio)
    
    if result:
        match, score = result[:2]
        word_overlap = len(set(query.split()) & set(match.split())) / max(len(query.split()), len(match.split()))
        
        if score >= 90 and word_overlap >= 0.7:  
            return match

    return None

# ** Apply Fuzzy Matching Only on Remaining Cases **
parallel_fuzzy_matches = Parallel(n_jobs=-1)(delayed(secondary_fuzzy_match)(
    STrain["filtered_name"].iloc[i],  
    [G["filtered_name"].iloc[idx] for idx in best_match_indices[i]]
) for i in range(len(STrain)))

# ** Step 7: Assign Best Matches Efficiently **
final_matches = []
final_scores = []

for i in range(len(STrain)):
    if best_match_scores[i][0] >= 0.8:
        final_matches.append(G["filtered_name"].iloc[best_match_indices[i][0]])
        final_scores.append(best_match_scores[i][0])
    elif parallel_phonetic_matches[i] is not None:
        final_matches.append(parallel_phonetic_matches[i])
        final_scores.append(0.85)
    elif parallel_fuzzy_matches[i] is not None:
        final_matches.append(parallel_fuzzy_matches[i])
        final_scores.append(0.88)
    else:
        final_matches.append(None)
        final_scores.append(-1)

# ** Step 8: Assign Company ID Efficiently **
G_dict = dict(zip(G["filtered_name"], G["company_id"]))  
STrain["matched_name"] = final_matches
STrain["best_match_score"] = final_scores
STrain["company_id_pred"] = STrain["matched_name"].map(G_dict).fillna(-1).astype(int)

# End timing
end_time = time.time()

# Print execution time
total_time = (end_time - start_time) / 60  
print(f"Final Optimized Execution Time: {total_time:.2f} minutes")


Final Optimized Execution Time: 13.91 minutes


In [15]:
# Apply cost function to each row
STrain["cost_function"] = STrain.apply(calculate_cost, axis=1)
# Count occurrences of each cost value (0, 1, 5)
cost_counts = STrain["cost_function"].value_counts().sort_index()

# Print the counts
print("Cost Function Occurrences:")
for cost, count in cost_counts.items():
    print(f"Cost {cost}: {count} occurrences")

# Optional: Convert to dictionary for further use
cost_distribution = cost_counts.to_dict()
print("\n Cost Distribution Dictionary:", cost_distribution)


Cost Function Occurrences:
Cost 0: 24441 occurrences
Cost 1: 23583 occurrences
Cost 5: 51976 occurrences

 Cost Distribution Dictionary: {0: 24441, 1: 23583, 5: 51976}


In [20]:
import time
import pandas as pd
import numpy as np
import dask.dataframe as dd
from joblib import Parallel, delayed
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from metaphone import doublemetaphone
from rapidfuzz import fuzz, process
from collections import Counter

# Load Data
G = pd.read_csv("G.csv", delimiter="|", encoding="utf-8")
STrain = pd.read_csv("STrain.csv", delimiter="|", encoding="utf-8")

# Start timing
start_time = time.time()

# ** Step 1: Name Standardization **
def standardize_company_name(name):
    """Normalize company names while selectively keeping legal structures."""
    if pd.isnull(name): return ""
    name = name.lower().strip()
    legal_terms = ["llc", "inc", "ltd", "s.p.a.", "s.r.l."]
    for term in legal_terms:
        if name.endswith(term):  
            continue  
        name = name.replace(term, "").strip()
    return name

G["clean_name"] = G["name"].map(standardize_company_name)
STrain["clean_name"] = STrain["name"].map(standardize_company_name)

# ** Step 2: High-Frequency Words (Reintroducing for Context) **
def get_high_freq_words(names, top_n=10):
    words = " ".join(names).split()
    word_counts = Counter(words)
    return [word for word, _ in word_counts.most_common(top_n)]

high_freq_words = get_high_freq_words(G["clean_name"], top_n=10)

def filter_common_words(name, common_words):
    """Remove common words to prevent overmatching, but only for initial match."""
    return " ".join([word for word in name.split() if word not in common_words])

G["filtered_name"] = G["clean_name"].map(lambda x: filter_common_words(x, high_freq_words))
STrain["filtered_name"] = STrain["clean_name"].map(lambda x: filter_common_words(x, high_freq_words))

# ** Step 3: TF-IDF Vectorization (Optimized) **
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1, 3), max_features=10_000)
tfidf_g_matrix = vectorizer.fit_transform(G["filtered_name"])
tfidf_s_matrix = vectorizer.transform(STrain["filtered_name"])

# ** Step 4: Nearest Neighbors Matching (`n_neighbors=2` for Precision) **
nn = NearestNeighbors(n_neighbors=2, metric="cosine", n_jobs=-1)
nn.fit(tfidf_g_matrix)

distances, best_match_indices = nn.kneighbors(tfidf_s_matrix, return_distance=True)
best_match_scores = 1 - distances  

# ** Step 5: Fuzzy Matching BEFORE Phonetic Matching (Higher Accuracy) **
def fuzzy_match(query, candidates):
    """Find best fuzzy match with strong structure matching."""
    result = process.extractOne(query, candidates, scorer=fuzz.token_sort_ratio)
    if result and result[1] >= 92:  # Higher threshold for correctness
        return result[0]
    return None

# Apply Fuzzy Matching First
parallel_fuzzy_matches = Parallel(n_jobs=-1)(delayed(fuzzy_match)(
    STrain["clean_name"].iloc[i],  
    [G["clean_name"].iloc[idx] for idx in best_match_indices[i]]
) for i in range(len(STrain)))

# ** Step 6: Phonetic Matching Only When Needed (Safer) **
phonetic_cache = {}

def get_phonetic_code(name):
    """Cache Metaphone codes to speed up computation."""
    if name in phonetic_cache:
        return phonetic_cache[name]
    code = doublemetaphone(name)[0]
    phonetic_cache[name] = code
    return code

def phonetic_match(query, candidates):
    """Phonetic matching using Metaphone with stricter rules."""
    query_code = get_phonetic_code(query)
    for name in candidates:
        if get_phonetic_code(name) == query_code:
            return name
    return "No Match"  # Instead of returning None

# Only process phonetic matching where fuzzy matching is None
indices_to_process = [i for i in range(len(STrain)) if parallel_fuzzy_matches[i] is None]

# Run Phonetic Matching in Parallel Only for Unmatched Cases
parallel_phonetic_matches = [None] * len(STrain)  # Initialize with None

phonetic_results = Parallel(n_jobs=-1)(
    delayed(phonetic_match)(
        STrain["clean_name"].iloc[i],  
        [G["clean_name"].iloc[idx] for idx in best_match_indices[i]]
    ) for i in indices_to_process
)

# Store results only for processed indices
for idx, i in enumerate(indices_to_process):
    parallel_phonetic_matches[i] = phonetic_results[idx]
# ** Step 7: Assign Best Matches (More Conservative Approach) **
final_matches = []
final_scores = []

for i in range(len(STrain)):
    if best_match_scores[i][0] >= 0.85:
        final_matches.append(G["clean_name"].iloc[best_match_indices[i][0]])
        final_scores.append(best_match_scores[i][0])
    elif parallel_fuzzy_matches[i] is not None:
        final_matches.append(parallel_fuzzy_matches[i])
        final_scores.append(0.92)
    elif parallel_phonetic_matches[i] is not None:
        final_matches.append(parallel_phonetic_matches[i])
        final_scores.append(0.85)
    else:
        final_matches.append(None)
        final_scores.append(-1)

# ** Step 8: Assign Company ID Efficiently **
G_dict = dict(zip(G["clean_name"], G["company_id"]))  
STrain["matched_name"] = final_matches
STrain["best_match_score"] = final_scores
STrain["company_id_pred"] = STrain["matched_name"].map(G_dict).fillna(-1).astype(int)

# End timing
end_time = time.time()

# Print execution time
total_time = (end_time - start_time) / 60  
print(f"Final Optimized Execution Time: {total_time:.2f} minutes")


Final Optimized Execution Time: 11.08 minutes


In [21]:
# Apply cost function to each row
STrain["cost_function"] = STrain.apply(calculate_cost, axis=1)
# Count occurrences of each cost value (0, 1, 5)
cost_counts = STrain["cost_function"].value_counts().sort_index()

# Print the counts
print("Cost Function Occurrences:")
for cost, count in cost_counts.items():
    print(f"Cost {cost}: {count} occurrences")

# Optional: Convert to dictionary for further use
cost_distribution = cost_counts.to_dict()
print("\n Cost Distribution Dictionary:", cost_distribution)


Cost Function Occurrences:
Cost 0: 26106 occurrences
Cost 1: 26340 occurrences
Cost 5: 47554 occurrences

 Cost Distribution Dictionary: {0: 26106, 1: 26340, 5: 47554}


In [151]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from metaphone import doublemetaphone
from rapidfuzz import fuzz, process

# Start timing
start_time = time.time()
# Load Data
G = pd.read_csv("G.csv", delimiter="|", encoding="utf-8")
STrain = pd.read_csv("STrain.csv", delimiter="|", encoding="utf-8")

# Step 1: Improved Name Standardization
def standardize_company_name(name):
    """Normalize company names while selectively keeping essential legal structures."""
    if pd.isnull(name):
        return ""

    name = name.lower().strip()
    
    # Retain meaningful legal structures
    legal_terms_to_keep = ["s.p.a.", "s.r.l.", "llc"]
    legal_terms_to_remove = ["gmbh", "inc", "s.a.", "sa", "ltd"]

    for term in legal_terms_to_remove:
        if term not in legal_terms_to_keep:
            name = name.replace(term, "").strip()

    return name

G["clean_name"] = G["name"].apply(standardize_company_name)
STrain["clean_name"] = STrain["name"].apply(standardize_company_name)

# Step 2: Enhanced TF-IDF Vectorization
vectorizer = TfidfVectorizer(
    stop_words="english",
    min_df=1,
    max_df=0.85,
    ngram_range=(1, 3),  # Increase n-gram range for better word combinations
    sublinear_tf=True,
    max_features=10_000
)

tfidf_g_matrix = vectorizer.fit_transform(G["clean_name"])
tfidf_s_matrix = vectorizer.transform(STrain["clean_name"])

# Step 3: Nearest Neighbors Search
nn = NearestNeighbors(n_neighbors=5, metric="cosine", n_jobs=-1)
nn.fit(tfidf_g_matrix)

# Step 4: Matching Logic
distances, best_match_indices = nn.kneighbors(tfidf_s_matrix, return_distance=True)
best_match_scores = 1 - distances  # Convert cosine distances to similarity scores

final_matches = []
final_scores = []

for i, match_list in enumerate(best_match_indices):
    top_match = None
    top_score = 0

    for j, idx in enumerate(match_list):
        similarity_score = best_match_scores[i][j]
        if similarity_score >= 0.8:  
            top_match = G["clean_name"].iloc[idx]
            top_score = similarity_score
            break  

    if not top_match:
        top_match = None
        top_score = -1

    final_matches.append(top_match)
    final_scores.append(top_score)

# Step 5: Improved Fuzzy Matching
def compute_similarity(query, candidates):
    """Ensure fuzzy matches have strong word overlap."""
    query_words = set(query.split())

    best_match, score = process.extractOne(query, candidates, scorer=fuzz.token_sort_ratio)

    if best_match:
        match_words = set(best_match.split())
        word_overlap = len(query_words & match_words) / max(len(query_words), len(match_words))

        if score >= 85 and word_overlap >= 0.7:
            return best_match

    return None

# Step 6: Final Matching & Filtering
G_dict = dict(zip(G["clean_name"], G["company_id"]))  # Precompute dictionary for fast lookup
STrain["matched_name"] = final_matches
STrain["best_match_score"] = final_scores
STrain["company_id_pred"] = STrain["matched_name"].map(G_dict).fillna(-1).astype(int)
# End timing
end_time = time.time()

# Print execution time
total_time = (end_time - start_time) / 60  # Convert to minutes
print(f" Final Optimized Execution Time: {total_time:.2f} minutes")


 Final Optimized Execution Time: 11.83 minutes


In [152]:
# Apply cost function to each row
STrain["cost_function"] = STrain.apply(calculate_cost, axis=1)
# Count occurrences of each cost value (0, 1, 5)
cost_counts = STrain["cost_function"].value_counts().sort_index()

# Print the counts
print("Cost Function Occurrences:")
for cost, count in cost_counts.items():
    print(f"Cost {cost}: {count} occurrences")

# Optional: Convert to dictionary for further use
cost_distribution = cost_counts.to_dict()
print("\n Cost Distribution Dictionary:", cost_distribution)


Cost Function Occurrences:
Cost 0: 23064 occurrences
Cost 1: 21949 occurrences
Cost 5: 54987 occurrences

 Cost Distribution Dictionary: {0: 23064, 1: 21949, 5: 54987}


In [155]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from rapidfuzz import fuzz, process

# Start timing
start_time = time.time()

# Load Data
G = pd.read_csv("G.csv", delimiter="|", encoding="utf-8")
STrain = pd.read_csv("STrain.csv", delimiter="|", encoding="utf-8")

# Step 1: Improved Name Standardization (Restoring Some Legal Terms)
def standardize_company_name(name):
    """Normalize company names while selectively keeping essential legal structures."""
    if pd.isnull(name):
        return ""

    name = name.lower().strip()
    
    # Keep necessary legal structures
    legal_terms_to_keep = ["s.p.a.", "s.r.l.", "llc", "inc", "ltd"]
    legal_terms_to_remove = ["gmbh", "s.a.", "sa"]

    for term in legal_terms_to_remove:
        if term not in legal_terms_to_keep:
            name = name.replace(term, "").strip()

    return name

G["clean_name"] = G["name"].apply(standardize_company_name)
STrain["clean_name"] = STrain["name"].apply(standardize_company_name)

# Step 2: Adjusted TF-IDF Vectorization
vectorizer = TfidfVectorizer(
    stop_words="english",
    min_df=1,
    max_df=0.85,
    ngram_range=(1, 3),  
    sublinear_tf=True,
    max_features=15_000  # Increase feature limit
)

tfidf_g_matrix = vectorizer.fit_transform(G["clean_name"])
tfidf_s_matrix = vectorizer.transform(STrain["clean_name"])

# Step 3: Nearest Neighbors Search (Stricter Filtering)
nn = NearestNeighbors(n_neighbors=3, metric="cosine", n_jobs=-1)  # Reduce neighbors from 5 to 3
nn.fit(tfidf_g_matrix)

# Step 4: Matching Logic
distances, best_match_indices = nn.kneighbors(tfidf_s_matrix, return_distance=True)
best_match_scores = 1 - distances  # Convert cosine distances to similarity scores

final_matches = []
final_scores = []

for i, match_list in enumerate(best_match_indices):
    top_match = None
    top_score = 0

    for j, idx in enumerate(match_list):
        similarity_score = best_match_scores[i][j]
        if similarity_score >= 0.9:  # Increased from 0.85 to 0.9 for stricter filtering
            top_match = G["clean_name"].iloc[idx]
            top_score = similarity_score
            break  

    if not top_match:
        top_match = None
        top_score = -1

    final_matches.append(top_match)
    final_scores.append(top_score)

# Step 5: More Strict Fuzzy Matching
def compute_similarity(query, candidates):
    """Ensure fuzzy matches have strong word overlap."""
    query_words = set(query.split())

    best_match, score = process.extractOne(query, candidates, scorer=fuzz.token_sort_ratio)

    if best_match:
        match_words = set(best_match.split())
        word_overlap = len(query_words & match_words) / max(len(query_words), len(match_words))

        if score >= 90 and word_overlap >= 0.8:  # Increased score threshold to 90 and overlap to 80%
            return best_match

    return None

# Step 6: Final Matching & Filtering
G_dict = dict(zip(G["clean_name"], G["company_id"]))  # Precompute dictionary for fast lookup
STrain["matched_name"] = final_matches
STrain["best_match_score"] = final_scores
STrain["company_id_pred"] = STrain["matched_name"].map(G_dict).fillna(-1).astype(int)

# End timing
end_time = time.time()

# Print execution time
total_time = (end_time - start_time) / 60  # Convert to minutes
print(f" Final Optimized Execution Time: {total_time:.2f} minutes")

 Final Optimized Execution Time: 20.42 minutes


In [156]:
# Apply cost function to each row
STrain["cost_function"] = STrain.apply(calculate_cost, axis=1)
# Count occurrences of each cost value (0, 1, 5)
cost_counts = STrain["cost_function"].value_counts().sort_index()

# Print the counts
print("Cost Function Occurrences:")
for cost, count in cost_counts.items():
    print(f"Cost {cost}: {count} occurrences")

# Optional: Convert to dictionary for further use
cost_distribution = cost_counts.to_dict()
print("\n Cost Distribution Dictionary:", cost_distribution)


Cost Function Occurrences:
Cost 0: 25653 occurrences
Cost 1: 31255 occurrences
Cost 5: 43092 occurrences

 Cost Distribution Dictionary: {0: 25653, 1: 31255, 5: 43092}


In [159]:
STrain = STrain.merge(G[['company_id', 'name', 'clean_name']], 
                      left_on='company_id_pred', 
                      right_on='company_id', 
                      how='left', 
                      suffixes=('', '_g'))
STrain[STrain["cost_function"] == 5].head(50)

,train_index,name,company_id,clean_name,matched_name,best_match_score,company_id_pred,cost_function,company_id_g,name_g,clean_name_g
0,0,ATRION Immo bilien & Co. KG,-1,atrion immo bilien & co. kg,gmk immo kg,1.000000,487148,5,487148.0,GMK Immo KG,gmk immo kg
2,2,Financial USI.,510805,financial usi.,nidaros financial,1.000000,135139,5,135139.0,Nidaros Financial S.A.,nidaros financial
3,3,FlexShares Trust - FlexShares Morningstar Emergingmarkets Factor Tilt Index Fund,523467,flexshares trust - flexshares morningstar emergingmarkets factor tilt index fund,flexshares trust - flexshares morningstar us market factor tilt index fund,0.965122,532081,5,532081.0,FlexShares Trust - FlexShares Morningstar US Market Factor Tilt Index Fund,flexshares trust - flexshares morningstar us market factor tilt index fund
6,6,"LLC TEBS Fund II, ATAX",277891,"llc tebs fund ii, atax","hillhouse fund ii, l.p.",0.943829,166143,5,166143.0,"Hillhouse Fund II, L.P.","hillhouse fund ii, l.p."
7,7,Windermere reds Acquisitions LLC,205639,windermere reds acquisitions llc,"trimax acquisitions, llc",1.000000,442737,5,442737.0,"Trimax Acquisitions, LLC","trimax acquisitions, llc"
8,8,Les Eoliennes de Saint Fraigne,-1,les eoliennes de int fraigne,apei les papillons blancs de int quentin,1.000000,411182,5,411182.0,APEI LES PAPILLONS BLANCS DE SAINT QUENTIN,apei les papillons blancs de int quentin
11,11,Enfo Oyj,9529,enfo oyj,gofore oyj,1.000000,176308,5,176308.0,Gofore Oyj,gofore oyj
12,12,Inteventor ApS,-1,inteventor aps,bering allerød aps,1.000000,359801,5,359801.0,Bering Allerød ApS,bering allerød aps
19,19,CAC,214855,cac,"l/n cac, llc",0.953514,634653,5,634653.0,"L/N CAC, LLC","l/n cac, llc"
25,25,Services Comitno,-1,services comitno,intor services,1.000000,112274,5,112274.0,Intor Services,intor services


Key Adjustments
1. Reduce Overmatching Bias:
- Penalize very common words (e.g., "trust", "fund", "partners") in TF-IDF.
- Require higher similarity thresholds (>= 0.93 instead of 0.85).
2.	Reintroduce Legal Terms Selectively:
- Restore "LLC", "Inc.", "Ltd." when they appear at the end of a name.
3.	Hybrid Matching Approach:
- First Matching Pass: TF-IDF + Nearest Neighbors
- Second Matching Pass: Jaccard Similarity (to check word overlap)
- Final Matching Pass: Fuzzy Matching (score >= 90)

In [11]:
# Save results
STrain[["train_index", "company_id_pred"]].to_csv("STrain_final_predictions.csv", sep="|", index=False)
